In [ ]:
import numpy as np
import re
import ast
import subprocess
import os, csv
from codes.utils.file import create_file_abs, create_parent_dir, dir_exists_abs, file_exists_absolute, join_dir, file_exists_current, file_remove_current
from codes.parameters import ASTRAL, NUMPY_EXTENSION, QUARTETS, SCRIPT_FOLDER, STATS_FOLDER, TRUE_FOLDER, WQFM, \
    COMPLETE, GENE_TREE, QUARTETS, INCOMPLETE, IMPUTED_LIST, OUTPUT_FOLDER, NUMPY_ARRAY, REMOVED_TAXA_RANGE, INPUT_FOLDER, \
    QUARTETS_EXTENSION, MODES_LIST, WORKING_FOLDER, METHODS, TERRACE_OUTPUT_FOLDER

In [ ]:
# from IPython import get_ipython

In [ ]:
terrace_dir = join_dir(TERRACE_OUTPUT_FOLDER)
consensus_dir = join_dir(terrace_dir, "consensus")
!rm -rf "$consensus_dir"
!mkdir "$consensus_dir"

if not dir_exists_abs(terrace_dir):
    print("Directory does not exist: {}".format(terrace_dir))
    exit(1)

In [ ]:
# get consensus trees
for terrace_file in os.listdir(terrace_dir):
    file_path = join_dir(terrace_dir, terrace_file)
    if file_path.endswith(".txt"):
        print("Processing file: {}".format(file_path))
        terrace_file_name = terrace_file.split(".")[0]
        !./bin/raxml-ng --consense MRE --tree "$file_path" --prefix "$consensus_dir"/"$terrace_file_name" 

In [ ]:
def remove_numbers_from_consensus_tree(file_path):
    file = open(file_path, "r")
    lines = file.readlines()
    line = lines[0]
    line = re.sub(r'[0-9]', '', line)
    # print(line)
    file = open(file_path, 'w')
    file.writelines(line)
    file.close()

In [ ]:
# get RF distance
def get_RF_distance(true_tree, file_abs):
    result = ast.literal_eval(subprocess.check_output(['getFpFn.py', '-t', true_tree, '-e', file_abs], encoding='UTF-8'))
    assert result[0] == result[1] == result[2]
    dist = str(result[0])
    # print( dist)
    return dist

In [ ]:
true_tree = join_dir(TRUE_FOLDER, 'true-species.out.tree')

In [ ]:
print("Removing numbers from consensus trees")
CONSENSUS_TREE_EXTENSION = "consensusTreeMRE"
csv_out_list = []
for consensus_file in os.listdir(consensus_dir):
    if consensus_file.endswith(CONSENSUS_TREE_EXTENSION):
        file_path = join_dir(consensus_dir, consensus_file)
        print("Processing file: {}".format(file_path))
        remove_numbers_from_consensus_tree(file_path)
        dist = get_RF_distance(true_tree, file_path)
        print("RF distance: {}".format(dist))
        missing_range = consensus_file.split(".")[0].split("_")[0]
        replica_no = consensus_file.split(".")[0].split("_")[1]
        csv_out_list.append([replica_no, missing_range, dist])
# print(csv_out_list)

In [ ]:
stats_file = "stats/RF_consensus.csv"
with open(stats_file, "w") as stat_file:
    csv.writer(stat_file).writerows(csv_out_list)